### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [ ]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

In [ ]:
from python.hpc import HybridArray

In [ ]:
import numpy as np
from python.analysis.analysis import single_heatmap_auc_vs_r_beta_range, multi_heatmap_auc_vs_r_beta_range

In [ ]:
N = 1000
num_monte = 1000
num_r = 21
num_beta = 21
r_range = np.linspace(1.0/num_r, 1.0, num_r)
beta_range = np.linspace(0.5,1.0,num_beta)
alpha = 0.1
print(f'{r_range=}')
print(f'{beta_range=}')

In [ ]:
single_heatmap_auc_vs_r_beta_range(N=N, num_monte=num_monte,\
            r_range=r_range, beta_range=beta_range,\
            alpha_selection_method = alpha,\
                transform_method='identity',\
            discovery_method='')

In [ ]:
single_heatmap_auc_vs_r_beta_range(N=N, num_monte=num_monte,\
            r_range=r_range, beta_range=beta_range,\
            alpha_selection_method = alpha,\
            transform_method='higher_criticism',\
            discovery_method='')

In [ ]:
single_heatmap_auc_vs_r_beta_range(N=N, num_monte=num_monte,\
            r_range=r_range, beta_range=beta_range,\
            alpha_selection_method = alpha,\
                transform_method='higher_criticism_unstable',\
            discovery_method='')

In [ ]:
single_heatmap_auc_vs_r_beta_range(N=N, num_monte=num_monte,\
            r_range=r_range, beta_range=beta_range,\
            alpha_selection_method = alpha,\
                transform_method='berk_jones',\
            discovery_method='')

In [ ]:
multi_heatmap_auc_vs_r_beta_range(r_range=r_range, beta_range=beta_range, N=N, num_monte=num_monte,\
                                  recipe=['identity',\
                                        ('higher_criticism','',alpha),\
                                        ('higher_criticism_unstable','',alpha),\
                                        ('berk_jones','',alpha)])